In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [5]:
def scrape_case_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the relevant section containing case details (this may vary based on website structure)
    cases = []
    
    # Example: Assuming cases are listed in <div class="case"> tags (you need to inspect the actual HTML)
    for case_div in soup.find_all('div', class_='case'):
        case_name = case_div.find('h2').text.strip()  # Adjust based on actual structure
        citation = case_div.find('span', class_='citation').text.strip()  # Adjust based on actual structure
        year = case_div.find('span', class_='year').text.strip()  # Adjust based on actual structure
        facts = case_div.find('p', class_='facts').text.strip()  # Adjust based on actual structure
        issues = case_div.find('p', class_='issues').text.strip()  # Adjust based on actual structure
        judgment_summary = case_div.find('p', class_='summary').text.strip()  # Adjust based on actual structure
        
        cases.append({
            'case_name': case_name,
            'citation': citation,
            'year': year,
            'facts': facts,
            'issues': issues,
            'judgment_summary': judgment_summary
        })
    
    return cases


In [6]:
def main():
    url = "https://www.indiankanoon.org/search/?formInput=constitution"  # Example URL (modify as needed)
    all_cases = scrape_case_data(url)

    # Create DataFrame and save to CSV
    df = pd.DataFrame(all_cases)
    df.to_csv('constitutional_cases_dataset.csv', index=False)
    print("Dataset saved as constitutional_cases_dataset.csv")

if __name__ == "__main__":
    main()


Dataset saved as constitutional_cases_dataset.csv


In [11]:
import requests
def get_cases(keyword, max_cases=10):
    url = f"https://api.indiankanoon.org/search/?formInput={keyword}"
    response = requests.get(url)
    # Parse response to extract case details
    return response


print(get_cases("Article 14"))

<Response [401]>


In [13]:
import requests
import pandas as pd
import time

def get_cases(keyword="constitution", max_cases=200):
    API_KEY = "eb9ac60185dc449a177c16a3f881476b16999a01"  # Replace with your API key
    headers = {"Authorization": f"Token {API_KEY}"}
    
    cases = []
    page = 1
    
    try:
        while len(cases) < max_cases:
            url = f"https://api.indiankanoon.org/search/?formInput={keyword}&pn={page}"
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                print(f"Error: API returned status code {response.status_code}")
                break
                
            data = response.json()
            
            if not data.get("results"):
                print("No more results found")
                break
                
            for result in data["results"]:
                cases.append({
                    "case_name": result.get("title", "N/A"),
                    "citation": result.get("citation", "N/A"),
                    "year": result.get("date", "N/A").split("-")[0] if result.get("date") else "N/A",
                    "judgment_summary": result.get("doc", "N/A")[:200] + "...",  # First 200 characters
                    "url": f"https://indiankanoon.org/doc/{result['tid']}/"
                })
                
                if len(cases) >= max_cases:
                    break
                    
            page += 1
            time.sleep(1)  # Rate limiting
            
        return pd.DataFrame(cases)
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return pd.DataFrame(cases)

# Example usage
if __name__ == "__main__":
    df = get_cases(keyword="article 14", max_cases=200)
    df.to_csv("indian_constitutional_cases.csv", index=False)
    print(f"Saved {len(df)} cases to CSV")


Error: API returned status code 405
Saved 0 cases to CSV


In [15]:
import requests
import pandas as pd
import time

def get_cases(keyword="constitution", max_cases=200):
    API_KEY = "eb9ac60185dc449a177c16a3f881476b16999a01"  # Replace with your actual API key
    headers = {"Authorization": f"Token {API_KEY}"}  # Using Token Auth
    
    cases = []
    page = 0  # pagenum starts at 0
    
    try:
        while len(cases) < max_cases:
            url = f"https://api.indiankanoon.org/search/?formInput={keyword}&pagenum={page}"  # SEARCH endpoint
            response = requests.post(url, headers=headers)  # POST request
            
            if response.status_code == 404:
                print("Error 404: Check URL, parameters, or API endpoint.")
                break
            
            if response.status_code == 403:
                print("Error 403: Authentication failed. Check API key format or status.")
                break
            
            if response.status_code != 200:
                print(f"Error {response.status_code}: {response.text}")
                break
            
            data = response.json()
            
            if not data.get("docs"):  # "docs" instead of "results"
                print("No more results found.")
                break
            
            for doc in data["docs"]:  # Use "docs" key
                cases.append({
                    "case_name": doc.get("title", "N/A"),
                    "citation": doc.get("citation", "N/A"),
                    "year": doc.get("publishdate", "N/A").split("-")[0] if doc.get("publishdate") else "N/A",
                    "url": f"https://indiankanoon.org/doc/{doc['tid']}/"
                })
                
            page += 1
            time.sleep(2)  # Rate limiting
            
        return pd.DataFrame(cases)
    
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        return pd.DataFrame(cases)

# Example
df = get_cases(keyword="constitution")
df.to_csv("cases.csv", index=False)


NameError: name '__file__' is not defined